In [11]:
import pandas as pd
import numpy as np
from collections import defaultdict
# from sklearn.preprocessing import StandardScaler
# from matplotlib import pyplot as plt

In [12]:
import os
os.getcwd()

'/Users/nikanizharadze/Desktop/AoA/BLE Ray-tracing Simulation Dataset'

In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [14]:
rooms = ['testbench_01', 'testbench_01_furniture_low', 'testbench_01_furniture_mid', 'testbench_01_furniture_high']
concrete_rooms = ['testbench_01_furniture_low_concrete', 'testbench_01_furniture_mid_concrete', 'testbench_01_furniture_high_concrete']
other_scenarios = ['testbench_01_rotated_anchors']
anchors = ['anchor1', 'anchor2', 'anchor3', 'anchor4']
channels = ['37','38','39']
polarities = ['V','H']

In [15]:
import json
import os
os.chdir('BLE Ray-tracing Simulation Dataset')
# os.getcwd()

FileNotFoundError: [Errno 2] No such file or directory: 'BLE Ray-tracing Simulation Dataset'

In [ ]:
#read data
data = defaultdict(lambda: defaultdict(lambda: defaultdict (lambda: defaultdict(list))))
anchor_data = defaultdict(lambda: defaultdict(lambda: defaultdict (lambda: defaultdict(list))))

for room in rooms + concrete_rooms + other_scenarios: 
    for channel in channels:  
        for polarity in polarities: 
             
            tag_filename = f'{room}/tag_ml_export_CH{channel}_{polarity}.json'
            tag_df = pd.read_json(tag_filename, orient='records')

            anchor_filename = f'{room}/anchor_ml_export_CH{channel}_{polarity}.json'
            anchor_df = pd.read_json(anchor_filename, orient='records')

            df = tag_df.merge(anchor_df)

            # remove calibration points
            df.drop(df[(df['x_tag']==0).values | (df['y_tag']==0).values | (df['z_tag']==0).values].index, inplace=True)
            
            for anchor in anchors:
                data[room][anchor][channel][polarity] = df[df['anchor']==int(anchor[-1])]
                anchor_data[room][anchor][channel][polarity] = anchor_df
                
                

In [16]:
df = pd.DataFrame()
final_df = pd.DataFrame()
for room in rooms:
    for anchor in anchors:
        for channel in channels:
            for polarity in ['V', 'H']:
                for k,v in data[room][anchor][channel][polarity].items():
                    temp = pd.DataFrame(v)
                    df = pd.concat([df, v],axis=1)
                # df['anchor_name'] = anchor
                df['polarity'] = polarity
                df['channel'] = channel
                df['room'] = room
                final_df = pd.concat([df, final_df],axis=0)
                df = pd.DataFrame()

In [20]:
final_df.shape

(234912, 24)

In [10]:
final_df.to_pickle('raw_IQ.pkl')

In [12]:
anchors = ['anchor1', 'anchor2', 'anchor3', 'anchor4']
channels = ['37','38','39']
polarities = ['V','H']

def iq_processing(data):
    
    """
    Input: Data
    Output: Processed Data
    Processing: Power Scaling, IQ shifting
    """

    cols_real = ['pdda_input_real_{}'.format(x+1) for x in range(5)]
    cols_imag = ['pdda_input_imag_{}'.format(x+1) for x in range(5)]

    iq_values = pd.DataFrame(data['pdda_input_real'].tolist(), columns=cols_real, index=data.index)
    iq_values[cols_imag] = pd.DataFrame(data['pdda_input_imag'].tolist(), columns=cols_imag, index=data.index)
    
    phase = pd.DataFrame(np.arctan2(iq_values['pdda_input_imag_1'],iq_values['pdda_input_real_1']), columns=['phase_1'])
    
    cos = np.cos(phase).values.ravel()
    sin = np.sin(phase).values.ravel()
    
    out = data.copy()
    iq_ref = np.abs(iq_values[f'pdda_input_real_1']*cos + iq_values[f'pdda_input_imag_1']*sin)
    for i in range(1,6):
        out[f'pdda_input_real_{i}'] = (iq_values[f'pdda_input_real_{i}']*cos + iq_values[f'pdda_input_imag_{i}']*sin)
        out[f'pdda_input_imag_{i}'] = (-iq_values[f'pdda_input_real_{i}']*sin + iq_values[f'pdda_input_imag_{i}']*cos)
        iq_ref +=  iq_values[f'pdda_input_real_{i}']**2 + iq_values[f'pdda_input_imag_{i}']**2

    power_norm =  StandardScaler().fit_transform((out['reference_power'] + out['relative_power']).values.reshape(-1,1))/10
    
    # out.insert(22, 'power', power_norm)
    # out.insert(21, 'iq_ref', iq_ref)
    out['power'] = power_norm
    out['iq_ref'] = iq_ref
    out.drop(columns=['pdda_input_imag_1', 'pdda_input_real', 'pdda_input_imag'], inplace=True)

    return out

In [13]:
df = iq_processing(final_df)

In [14]:
df.sort_values(['point', 'anchor', 'room', 'channel'], inplace=True)
# df.reset_index(inplace=True)

In [15]:
df = df.sort_values(['point', 'anchor', 'channel', 'room'])
df['mask'] = df['reference_power'] + df['relative_power']
df = df.sort_values(['point', 'anchor', 'channel', 'room', 'mask'])
# select to use either Vertical or Horizontal polarity based on reference and relative power
df = df.drop_duplicates(subset=['point', 'anchor', 'channel','room'], keep='first')

In [1]:
df.describe

NameError: name 'df' is not defined

In [72]:
df.to_pickle('df.pkl')

In [19]:
df.shape

(117456, 34)